# MNIST For ML Beginners
https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/index.html#mnist-for-ml-beginners

# MNIST データをダウンロード

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 手書き文字画像はこんな感じ
28px * 28px のグレースケール画像。<br>
今回のタスクは、この画像を入力としてどの数字であるかを識別すること。
<img src="MNIST_For_ML_Beginners/MNIST.png" width="500px">
このままじゃコンピュータは読めないので、、、値域0~1で符号化します
<img src="MNIST_For_ML_Beginners/MNIST-Matrix.png" width="500px">


### データセットの構成
学習用画像データセット：55,000枚 (mnist.train)<br>
テスト用画像データセット：10,000枚 (mnist.test)<br>

## モデルにどう入力するか、モデルからどう出力するか

### 入力の設計
今回は28行28列のベクトルを、単純に784行1列のベクトルに変換して入力ベクトルとする。
<img src="MNIST_For_ML_Beginners/mnist-train-xs.png" width="500px">

### 出力(正解ラベル)の設計
問題<br>
識別器は何を出力したら、良さげでしょうか？？<br>
その場合、正解ラベルはどのように表現すればいいでしょうか？？<br>

ベストプラクティス<br>
出力：各クラスに識別される確率の分布<br>
正解ラベル：one-hot-vector

識別クラス数と同じ要素数のベクトルで、クラスkが正解である出力ベクトルは、k番目が「1」で、その他が「0」をとるone-hot-vector <br>
例えば「3」が正解であるベクトルは [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]T
<img src="MNIST_For_ML_Beginners/mnist-train-ys.png" width="500px">

# 手書き文字を表現するためのモデルを考える
今回は、モデルは２層のニューラルネットワークを扱います。（※これはディープラーニングではありません。）<br>

### ニューラルネットワークの構造
ニューラルネットワークの構造は以下のとおり（簡単化のために、図は入出力ともに３次元ベクトルで表現している）。<br>
「ユニット」と、それらを結ぶ「エッジ」で構成されている。
<img src="MNIST_For_ML_Beginners/softmax-regression-scalargraph.png" width="500px">

（今回の）ニューラルネットワークは入力層・出力層の２層構造で構成されている。<br>
まず入力層のユニットからデータ「x」(784行1列のベクトルデータ)が入力され、エッジを通して次の層のユニットへと伝搬される。<br>
それを繰り返して出力層のユニットから最終的な出力「y」(10行1列のベクトルデータ)が得られる。<br>

### 減衰
エッジにはそれぞれ重み「w」がついており、エッジを通るデータはこのwによって重み付けをされる。<br>
さらにバイアス「b」が加算され、次の層のユニットへと伝搬する。<br>
また、各ユニットでは活性化関数と呼ばれる関数により、伝搬してきたデータを変換してから次のエッジへ送る。<br>

### 活性化関数
今回のモデルでは、入力層のユニットの活性化関数として恒等関数、出力層のユニットの活性化関数としてsoftmax関数を用いている。<br>
恒等関数とは、各ユニットに入力された値をそのまま出力する関数である。<br>
softmax関数は、各ユニットに入力された値を「0~1」の範囲に変換し出力する関数である。<br>
さらに、出力層の全ユニットから得られる出力の総和が「1」となるように調整される。<br>
softmax関数により、各ユニットから出力される値を、そのユニットが表す識別クラスである「確率」とみなすことができるようになる。<br>
（この考え方は、ロジスティック回帰の2値分類を、多クラス分類へと拡張させたものである。）<br>

これらは人間の脳神経における情報の伝搬と記憶の原理をモデル化したものであるため、ニューラルネットワーク（神経回路）モデルと呼ぶ。<br>

### 数式に落とし込む
入力xとエッジの付けられた重みw、活性化関数(softmax関数)、出力yを用いると、上記のモデルは以下の様に表現できる。
<img src="MNIST_For_ML_Beginners/softmax-regression-scalarequation.png" width="500px">

これは行列と（列）ベクトルを用いて、以下の様に表現できる。
<img src="MNIST_For_ML_Beginners/softmax-regression-vectorequation.png" width="500px">

重みwの行列を「W」とすれば、以下の様な式で書ける。<br>
$y = \text{softmax}(Wx + b)$

# 実装しちゃおう！

### まずはモデルの定義から
以上で述べたモデルは、Tensor Flowを使えば簡単に書けちゃいます<br>
まずは、ニューラルネットワークを構成する入力と出力について定義する。

In [2]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

続いて、重みとバイアスを定義する。

In [3]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

そして、さきほど説明した数式$y = \text{softmax}(Wx + b)$を作る。

In [4]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

### 誤差を定義しよう！（cross-entropy関数）

これで推定値yを求めることできる道具と、手元には正解ラベルy_が揃いました・・・つまり、学習できる！！！

学習するとは、「推定値と正解の誤差（誤差関数、損失関数）を最小化するような、重みwとバイアスbを求める」ということ。<br>
なので、学習するためには、まず誤差を定義してあげなきゃだめ<br>

今回は、損失関数にcross-entropy関数を用いる。<br>
yを推定値（１０行１列の確率分布）、y'を正解ラベル（１０行１列のone-hot-vector）とすると、<br>
$H_{y'}(y) = -\sum_i y'_i \log(y_i)$

なぜcross-entropy関数を損失関数として使っているのかについては、下記本のp.16-p.20にわかりやすく書かれているので気になる人はチェック！<br>
ロジスティック回帰と同じ仕組みなので、それから勉強すると、good<br>

「深層学習」岡谷貴之、機械学習プロフェッショナルシリーズ<br>
https://www.amazon.co.jp/dp/B018K6C99A/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1

<img src="MNIST_For_ML_Beginners/dl.jpg">

In [5]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


### 誤差を最小化したい！勾配降下法

誤差を定義できたら、いよいよそれを最小化してあげれば良い。<br>
ニューラルネットワークには、この誤差を最小化する方法が確立されている。<br>
勾配降下法と呼ばれるものだ。<br>

Tensor Flowには、それを計算してくれる関数が用意されてるので、さっきの誤差関数をぶっ込めばok！！！<br>

理解したくば、「深層学習」p.23-p.54ページをなめ回すように読もう！（３章：確率的勾配降下法、４章：誤差逆伝搬法）<br>

このような最適化アルゴリズムはdeep learningのような学習モデルとは独立したもの。<br>
deep learningを設計する上ではあまり考えなくても良いと思います。<br>


In [9]:
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

learning_rateとは「１回データを与えた時の学習効果をどれほどにするか」という調整を行うハイパーパラメータ。<br>
あの人は話を盛るから話半分くらいに受け止めよう、みたいなことを実現するもの

### 学習しよう！

もう手札は揃った。
いざ学習じゃあああ！

ところで、一気に全てのデータを使って学習する（バッチ学習）のは計算が大変なんだよね。。<br>
小さいデータセットを作って学習する（ミニバッチ学習）方法をとるのが効率的！<br>
これを確率的勾配降下法と呼びます。確率的なのは、小さいデータセットを作る時ランダムにデータを選んでいるから。<br>
ミニバッチのサイズは10~100が良いとされている。

In [10]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

100個のデータを使った学習を、1000回繰り返し行っています。<br>
ディープラーニングでは、この繰り返し回数を「エポックサイズ」と呼んでいます。<br>
重みや損失関数の収束具合を見て適切なエポックサイズを選んであげる必要があります。<br>

### 結果はどうなのよ
学習したモデルを評価しましょう。<br>

当然ですが、どのように評価するかをまず定義しなければなりません。<br>
１行目の式は、推定値と正解ラベルのargmaxが一致しているかどうかをbooleanで返す、というものを定義しています。<br>
２行目で、正解率を算出するものを定義しています。<br>
３行目で、実際にテストデータを使って計算しています。

In [8]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9151


# 約92% !!!

めでたし。めでたし。

、、、とはならない！！！

続く、チュートリアルにはこの結果について以下のように書かれています。<br>
「Getting 92% accuracy on MNIST is bad. It's almost embarrassingly bad.」<br>

## （意訳）「92%程度で喜んでんじゃねーぞks!」

ということなので、続くチュートリアルをこなして、99%台を目指しましょう！